## Data Cleaning & Extraction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read files 
jan = pd.read_csv("Resources/201901-citibike-tripdata.csv")
feb = pd.read_csv("Resources/201902-citibike-tripdata.csv")
mar = pd.read_csv("Resources/201903-citibike-tripdata.csv")
apr = pd.read_csv("Resources/201904-citibike-tripdata.csv")
may = pd.read_csv("Resources/201905-citibike-tripdata.csv")
jun = pd.read_csv("Resources/201906-citibike-tripdata.csv")
jul = pd.read_csv("Resources/201907-citibike-tripdata.csv")
aug = pd.read_csv("Resources/201908-citibike-tripdata.csv")
sep = pd.read_csv("Resources/201909-citibike-tripdata.csv")
oct = pd.read_csv("Resources/201910-citibike-tripdata.csv")
nov = pd.read_csv("Resources/201911-citibike-tripdata.csv")
dec = pd.read_csv("Resources/201912-citibike-tripdata.csv")

Check whether we need to remove any records,and it shows that only start & end stations are missing some records.
I keep it as it is not wrong records but just missing information, and it would not impact results of station analysis as
it is just a small scale.


In [ ]:
df_list = [jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec]
for df in df_list:
    print(f"There are {df.shape[0]} row")
    print(df.isna().sum())
    print("="*40)

In [ ]:
# Filter the columns that we want & add Month column
drop_columns = ["stoptime","end station id","end station latitude","end station longitude","bikeid"]

for df in df_list: 
    df.drop(columns = drop_columns, inplace=True)
    df["month"]= [time[5:7]for time in df["starttime"]]

# Make the final data for analysis
ana_df = pd.concat(df_list, axis=0, ignore_index=True)
ana_df.head()

In [ ]:
# Save it for further analysis
ana_df.to_csv("Resources/2019_ana.csv")

## Analysis 1:　Ｍonthly total trips (by customer type)

In [ ]:
# ana_df["usertype"].unique()
mon_total = ana_df[["usertype", "month"]]
x = mon_total["month"].unique()
y_sub = mon_total.loc[mon_total["usertype"]=="Subscriber"].groupby("month").count()["usertype"]/1000000
y_cus = mon_total.loc[mon_total["usertype"]=="Customer"].groupby("month").count()["usertype"]/1000000

In [ ]:
plt.figure(figsize=[10,6])
plt.plot(x, y_sub, "o-c", label="Subscriber")
plt.plot(x, y_cus, "o-g", label="Customer")
plt.legend(loc="upper left")
plt.ylim(0, max(y_sub+0.2))
plt.title("Monthly Total Trips", fontsize=20)
plt.ylabel("Total Trips (MM)", fontsize=15)
plt.savefig("analysis/1.jpg")
plt.show()

## Analysis 2: Top Age Groups Utilising Program

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

ana_df = pd.read_csv("Resources/2019_ana.csv")

In [2]:
from datetime import datetime
age_group = ana_df[["usertype", "birth year"]]
age_group["age"]= datetime.now().year - age_group["birth year"]

C:\Users\Eva\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
print(age_group["age"].min(), age_group["age"].max())
print(age_group["age"].value_counts().to_string())

19 165
53     1711935
33      853933
32      850747
31      817709
30      803898
34      801123
29      740948
35      722834
36      668711
28      656337
37      654738
38      606984
39      562845
27      556999
40      519622
41      489789
42      465719
43      430920
44      372967
26      360156
45      358417
46      356855
52      350497
47      347136
48      325352
51      324270
50      320960
49      313655
54      293922
55      287708
57      268630
56      268051
58      260313
59      243358
25      238771
60      231507
62      202959
61      196914
24      176719
63      167549
23      161943
64      148515
65      133757
22      124106
66      122537
67       98021
68       90211
69       78878
21       62453
70       49639
71       47903
72       36460
73       28572
74       27286
20       27089
75       24376
76       18676
77       11990
19       10291
80        9663
79        9336
78        8295
81        6818
82        4838
85        3595
122       3430
86 

In [4]:
age_group.loc[age_group["age"]>120].shape

(8601, 3)

In [ ]:
bins = [0,20,30,40,50,60,70,80,200]
labels = ["under 20","20-30","30-40","40-50","50-60","60-70","70-80", "Other"]
age_group["age_group"]=pd.cut(x=age_group['age'],bins=bins, labels=labels)
age_group.head()

In [ ]:
# get plotting data
y_sub = age_group.loc[age_group["usertype"]=="Subscriber"].groupby("age_group").count()["usertype"] / 1000
y_cus = age_group.loc[age_group["usertype"]=="Customer"].groupby("age_group").count()["usertype"] / 1000
base = age_group.groupby("age_group").count()["usertype"]
per = y_sub*1000/base
label = [str(per[i]*100)[:4]+"%" for i in range(len(per))]

In [ ]:
plt.figure(figsize=[10,8])
plt.bar(labels, y_sub,color="c", label="Subscriber (with % of respective age groups at the top)")
plt.bar(labels, y_cus, bottom=y_sub, color="g", label="Customer")
plt.title("Total Trips by Age Groups", fontsize=20)
plt.ylabel("Trip No. (K)", fontsize=15)
plt.legend(loc="upper right")
for i in range(len(label)):
    plt.text(labels[i], (y_sub[i]+y_cus[i]+100), label[i], ha="center")
plt.savefig("analysis/2.jpg")
plt.show()

## Analysis 3: Total Trips by Gender

In [ ]:
ana_df.head()

In [ ]:
gender_df = ana_df[["usertype", "gender"]]
x_label=["Unknown", "Male","Female"]
y_sub = gender_df.loc[gender_df["usertype"]=="Subscriber"].groupby("gender").count()["usertype"] / 1000
y_cus = gender_df.loc[gender_df["usertype"]=="Customer"].groupby("gender").count()["usertype"] / 1000
base = gender_df.groupby("gender").count()["usertype"]
per = y_sub*1000/base
label = [str(per[i]*100)[:4]+"%" for i in range(len(per))]

In [ ]:
plt.figure(figsize=[10,5])
plt.barh(x_label, y_sub,color="c", label="Subscriber (with % of respective age groups at the right)")
plt.barh(x_label, y_cus, left=y_sub, color="g", label="Customer")
plt.title("Total Trips by Gender", fontsize=20)
plt.xlabel("Trip No. (K)", fontsize=15)
plt.legend(loc="upper right")
for i in range(len(label)):
    plt.text((y_sub[i]+y_cus[i]+450),x_label[i],label[i], ha="center")
plt.savefig("analysis/3.jpg")
plt.show()

## Analysis 4: Peak Hours In A Day


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ana_df = pd.read_csv("Resources/2019_ana.csv")
ana_df.head()

In [ ]:
from datetime import datetime
time_table = ana_df[["starttime", "birth year"]]
time_table["time"] = [time_table["starttime"][i][11:13] for i in range(time_table.shape[0])]
time_table["age"] = datetime.now().year - time_table["birth year"]
bins = [0,20,30,40,50,60,70,80,200]
labels = ["under 20","20-30","30-40","40-50","50-60","60-70","70-80", "Other"]
time_table["age_group"]=pd.cut(x=time_table['age'],bins=bins, labels=labels)
time_table.head()

In [ ]:
x = time_table["time"].unique()
y = []

for group in labels:
    y.append(time_table.loc[time_table["age_group"] == group].groupby("time").count()["age_group"]/1000)

In [ ]:
plt.figure(figsize=[12,8])
plt.stackplot(x,y,labels=labels, alpha=0.8)
plt.title("Hourly Total Trips", fontsize=20)
plt.ylabel("Trip No. (K)", fontsize=15)
plt.xlim([0,23])
plt.legend(loc="upper left")
plt.grid(axis="x",linestyle="--",color="grey")
plt.savefig("analysis/4.jpg")
plt.show()

## Analysis 5: Trip Duration Analysis

In [1]:
import pandas as pd
ana_df = pd.read_csv("Resources/2019_ana.csv")
ana_df.head()

,Unnamed: 0,tripduration,starttime,start station id,start station name,start station latitude,start station longitude,end station name,usertype,birth year,gender,month
0,0,320,2019-01-01 00:01:47.4010,3160.0,Central Park West & W 76 St,40.778968,-73.973747,W 89 St & Columbus Ave,Subscriber,1971,1,1
1,1,316,2019-01-01 00:04:43.7360,519.0,Pershing Square North,40.751873,-73.977706,E 39 St & 2 Ave,Subscriber,1964,1,1
2,2,591,2019-01-01 00:06:03.9970,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,E 77 St & 3 Ave,Subscriber,1987,1,1
3,3,2719,2019-01-01 00:07:03.5450,504.0,1 Ave & E 16 St,40.732219,-73.981656,W 15 St & 6 Ave,Subscriber,1990,1,1
4,4,303,2019-01-01 00:07:35.9450,229.0,Great Jones St,40.727434,-73.993790,E 20 St & Park Ave,Subscriber,1979,1,1


In [ ]:
station_df = ana_df[["tripduration", "start station name", "start station latitude", "start station longitude"]]

# to understand the distribution of trip duration
# quantitively determine whether outliers exist using InterQuatil Range
quantile = station_df["tripduration"].quantile([0.25, 0.5, 0.75])
lowerq = quantile[0.25]
upperq = quantile[0.75]
IQR = upperq - lowerq
lower_bound = lowerq - 1.5*IQR
upper_bound = upperq + 1.5*IQR
upper_df = station_df.loc[station_df["tripduration"]>=upper_bound]
lower_df = station_df.loc[station_df["tripduration"]<=lower_bound]

In [ ]:
print(f"There are {upper_df.shape[0]/station_df.shape[0]} extreme larger values & {lower_df.shape[0]/station_df.shape[0]} extreme smaller values")

In [ ]:
print(upper_bound, station_df["tripduration"].max())

Analyse relatively short trips:

In [ ]:
within_1 = station_df.loc[(station_df["tripduration"]<3600)]
within_24 = station_df.loc[(station_df["tripduration"]<86400) & (station_df["tripduration"]>=3600)]

In [ ]:
print(within_24.shape[0]/station_df.shape[0])

In [ ]:
import numpy as np
from matplotlib.ticker import FuncFormatter

def mil_format(x, pos):
    return f"{x/1000000}"

fig,(ax1, ax2) = plt.subplots(1,2,figsize=(14,6))
fig.suptitle("Trip Duration Distribution (s)", fontsize=15)

# 10min (600s) as a bin
bins=np.arange(0,4200,600)
ax1.hist(within_1["tripduration"],bins=bins)
ax1.set_xticks(bins)
ax1.set_xlim([0,3600])
ax1.set_title("within 24 hrs")
ax1.set_ylabel("Trip No. (MM)", fontsize=15)
ax1.yaxis.set_major_formatter(FuncFormatter(mil_format))

# 1hr (3600s) as a bin
bins_over = np.arange(3600,90000,3600)
ax2.hist(within_24["tripduration"],bins=bins_over)
ax2.set_xticks(bins_over)
ax2.set_xlim([3600,86400])
ax2.set_title("between 1hr and 1d")
ax2.yaxis.set_major_formatter(FuncFormatter(mil_format))
plt.xticks(rotation=90)
plt.savefig("analysis/5.jpg")
plt.show()

Understand extreme long trips
* how many are ther
* which stations (circle indiate times & color indicate durations)

In [2]:
over_24_df = ana_df[["tripduration", "start station name", "start station latitude", "start station longitude"]].loc[ana_df["tripduration"]>=86400]
print(f"There are {over_24_df.shape[0]/ana_df.shape[0]} records with trip duration over 1 day.")

There are 0.00029768831255151337 records with trip duration over 1 day.


In [3]:
over_24_df["start station name"].nunique()

786

In [6]:
import gmaps
from config import gkey
gmaps.configure(api_key=gkey)
layout = {'width': '800px','height': '800px'}

location = over_24_df[["start station latitude", "start station longitude"]]
layer = gmaps.heatmap_layer(location, weights=over_24_df["tripduration"])

In [7]:
base = gmaps.figure(layout=layout)
base.add_layer(layer)
base

Figure(layout=FigureLayout(height='800px', width='800px'))